In [20]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter
from PIL import Image
from matplotlib.pyplot import specgram
from matplotlib.mlab import window_hanning
import scipy
import cv2
%reload_ext autoreload
%autoreload 2
# this is the main library used (sits on top of PyTorch)
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from sklearn import metrics

In [14]:
import pandas as pd
import numpy as np

In [23]:
# data file
dEDF = pd.read_csv('dEDF.csv').values
# time file
tEDF = pd.read_csv('tEDF.csv').values

In [24]:
Fs = 1/(tEDF[3]-tEDF[2])
Fs = Fs[0]
winSize = int(np.round(Fs))
winStep = int(np.round(Fs/2))
movingwin    = [0.200,0.005] # Window size, Step size [s]
iWin   = int(np.round(movingwin[0]*Fs/2)*2);
iStep  = int(np.round(movingwin[1]*Fs/2)*2);
noverlap = iWin-iStep
i_stop = 0
i = 0
counter = 0

In [25]:
def hannspecgramc(d1,movingwin,Fs,tEDF,flo,fhi):
    iStart = 1;
    iWin   = int(np.round(movingwin[0]*Fs/2)*2);
    iStep  = int(np.round(movingwin[1]*Fs/2)*2);
    T      = iWin/Fs;
    df     = 1/T;
    fNQ    = Fs/2;
    freq   = range(0,int(iWin/2+1))*df;
    counter=1;
    S      = np.zeros([int(np.ceil(len(tEDF)/iStep)), int(iWin/2+1)]);
    times  = np.zeros([int(np.ceil(len(tEDF)/iStep)), 1]);
    while iStart+iWin < len(tEDF):
        dnow    = d1[iStart:iStart+iWin-1];
        dnow    = dnow - np.mean(dnow);
        dnow    = np.hanning(len(dnow))*np.squeeze(dnow);
        spectrum= np.real(scipy.fft.fft(dnow)*np.conjugate(scipy.fft.fft(dnow)));
        S[counter,:]   = spectrum[:int(iWin/2+1)];
        times[counter] = tEDF[int(iStart+iWin/2)];

        counter=counter+1;
        iStart = iStart + iStep;
    S = S[:counter-1,:];
    times = times[:counter-1];
    S = S[:,(freq>=flo)&(freq<=fhi)]
    freq = freq[(freq>=flo)&(freq<=fhi)]
    return S,times,freq

In [28]:
start_times = []
stop_times = []

In [29]:
while tEDF[i_stop] < 600: 
#only looking at the first 600 seconds, can change to any value

    i_start = i
    i_stop  = i_start + winSize - 1

    dspec = dEDF[i_start:i_stop]
    dspec = dspec - np.mean(dspec)
    t     = tEDF[i_start:i_stop]
    flo = 30
    fhi = 250
    movingwin    = [0.200,0.005] # Window size, Step size [s]
    S0,S_times,S_freq = hannspecgramc(dspec,movingwin,Fs,t,flo,fhi)
    #Smooth the spectra.
    t_smooth = 11
    dt_S     = S_times[2]-S_times[1]
    S_smooth = gaussian_filter(S0,1)

    A = np.log10(S_smooth)
    A = np.flipud(A)
    fig, ax = matplotlib.pyplot.subplots(figsize=(6,6))
    ax.imshow(A,cmap='jet',
           extent = [min(S_freq),max(S_freq),min(S_times)[0],max(S_times)[0]],
             aspect = 'auto')

    name = 'demo_data/test/img_'+ str(counter)+'.jpg'
    
    ax.set_axis_off()
    fig.savefig(name)
    
    start_times.append(tEDF[i_start])
    stop_times.append(tEDF[i_stop])
    
    i = i_start + winStep;
    counter=counter+1;

NameError: name 'scipy' is not defined

In [ ]:
# where the data is
PATH = "data/demo_data"
# using resnet architecture
arch = resnet34
# size of square image in pixels
sz = 44
# transforms used on training data
transforms_up_down = [RandomScale(sz,1.2),RandomRotate(1)]
tfms = tfms_from_model(arch,sz,crop_type = CropType.NO,aug_tfms=transforms_up_down)
# data: comes from PATH, used tfms on training data, bs of 8 for training data, test data located in test folder
data = ImageClassifierData.from_paths(PATH,tfms=tfms,bs=8,test_name='test')
# load in pretraine
state = torch.load('saved_model.pkl',map_location=torch.device('cpu')) # remove map_location parameter if on GPU
learn2 = ConvLearner.pretrained(arch,data,precompute=False)
learn2.model.load_state_dict(state)

In [ ]:
log_preds_test = learn2.predict(is_test=True)
preds_test = np.argmax(log_preds_test,axis=1)
probs_test = np.exp(log_preds_test[:,1])

In [ ]:
test_names = np.empty_like(data.test_ds.fnames)
for i in range(len(data.test_ds.fnames)):
    test_names[i] = data.test_ds.fnames[i]
    #temp = data.test_ds.fnames[i]
    #matchobj = re.search('.*im.*',temp)
    #test_names[i] = matchobj.group()
test_df = pd.DataFrame(data = test_names,columns = ['image_number'])
test_df['prediction'] = preds_test
test_df['probability'] = probs_test

In [ ]:
test_df